In [1]:
#importing dependecies
import pandas as pd
from psaw import PushshiftAPI 
import datetime as dt
from config import db_password
from config import db_address
from sqlalchemy import create_engine

In [2]:
api = PushshiftAPI()

In [3]:
#create function to scrape reddit by date
def ticker_reddit_scraper(ticker,year,month,day):
    #querie reddit for ticker
    gen = api.search_comments(q=ticker, 
                              subreddit='wallstreetbets',
                              after=int(dt.datetime(year, month, day).timestamp()),
                              before=int(dt.datetime(year, month, day+1).timestamp()),
                              filter=['body','author','subreddit']
                             )
    #max allowed retrieved responses to ensure function doesn't take too long
    max_response=10000
    cache=[]
    for i in gen:
        #take only the dictionary from queries
        cache.append(i[-1])
        if len(cache) >= max_response:
            break
    df = pd.DataFrame(cache)
    if df.empty != True:
        try:
            df['created_utc']=pd.to_datetime(df['created_utc'],unit='s').dt.date
        except KeyError:
            return df
        df = df.drop(columns=['created'])
        df.rename(columns={'author':'username','created_utc':'date'}, inplace=True)
        df=df[['date','body','username','subreddit']]
    return df

In [4]:
#list of tickers
tickers=['BB', 'CRSR', 'TG', 'WEBR', 'ABNB', 'OI', 'CHGG', 'INTC', 'HP', 'SQ',
         'ROKU', 'PT', 'VXX', 'BYND', 'JBLU', 'GBTC',
         'MS', 'DKNG', 'FCF', 'GE', 'BJ', 'LCID',  'CS', 'KO',
         'WTI', 'RIG', 'RC', 'BA', 'MMAT', 'RKLB', 'UNH', 'RE', 
         'CSCO', 'VC', 'ZM', 'TIL', 'IBKR', 'CRM', 'IBM', 'ET', 'CP',
         'MSTR', 'DAC', 'DTC', 'NYT', 'TH', 'BP', 'LMT', 'SM', 
         'ITT', 'BBBY', 'TTD', 'PBR', 'SPCE', 'UPST', 'AFRM', 'NG']

In [5]:
#Because the scraper took so long I had to occasionaly interupt the scraping
#so as not to redo completed tickers this list was made
tickers_done=['GME', 'TSLA', 'AAPL', 'TWTR', 'ATER', 'AMC','AMD', 
         'MSFT', 'HD', 'FB', 'AMZN', 'NU', 'DTE', 'GOOG',
         'NFLX', 'SOFI', 'TA', 'BBIG', 'NVDA', 'AI', 'ZIM','VTNR',
         'PLTR', 'RIVN', 'LUNA', 'TTWO', 'OXY', 'BABA',  'WMT',
         'DWAC', 'CC', 'COO', 'LFG', 'RBLX', 'DIS', 'SE', 'SNDL', 'PARA',
         'SP', 'XOM', 'ES', 'JD', 'PTON', 'WBD', 'TGT', 'OG',
         'DM', 'DOW', 'TLRY', 'NIO', 'PANW', 'CVNA', 'JPM', 'IP', 'TD', 'GS', 
         'CLOV', 'PYPL', 'GM', 'IQ','MCD', 'MULN', 'DE', 'VZ', 'WW', 
         'EA']

In [30]:
#initial test scraping only gets the first 12 days
years = [2021,2020,2019,2018]
months=[1,2,3,4,5,6,7,8,9,10,11,12]
for ticker in tickers:
    for year in years:
        for month in months:
            for day in months:
                csv = ticker_reddit_scraper(ticker,year,month,day)
                if csv.empty != True:
                    #writes scraped data to csv named ticker_year_month_day
                    csv.to_csv('csv/'+ticker+'_'+str(year)+'_'+str(month)+'_'+str(day)+'.csv')

C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 524
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 522
  warnings.warn("Got non 200 code %s" % response.status_code)


In [7]:
#gets the rest of the majority of the month except the ends of the month
#due to an error in how the scraper function was written
years = [2021,2020,2019,2018]
months=[1,2,3,4,5,6,7,8,9,10,11,12]
days = [13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]
for ticker in tickers:
    for year in years:
        for month in months:
            for day in days:
                csv = ticker_reddit_scraper(ticker,year,month,day)
                if csv.empty != True:
                    csv.to_csv('csv/'+ticker+'_'+str(year)+'_'+str(month)+'_'+str(day)+'.csv',index=False)

C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 502
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 522
  warnings.warn("Got non 200 code %s" % response.status_code)


In [8]:
#rewrite the scraper function so the ends of the month can be taken
#removes the API call which 
def ticker_reddit_scraper_ends(gen):
    
    #max responses retrieved
    max_response=10000
    cache=[]
    for i in gen:
        #take only the dictionary from queries
        cache.append(i[-1])
        if len(cache) >= max_response:
            break
    df = pd.DataFrame(cache)
    if df.empty != True:
        try:
            df['created_utc']=pd.to_datetime(df['created_utc'],unit='s').dt.date
        except KeyError:
            return df
        df = df.drop(columns=['created'])
        df.rename(columns={'author':'username','created_utc':'date'}, inplace=True)
        df=df[['date','body','username','subreddit']]
    return df

In [11]:
#gets the ends of the month
years = [2021,2020,2019,2018]
months=[1,2,3,4,5,6,7,8,9,10,11,12]
days = [28,29,30,31]
for ticker in tickers:
    for year in years:
        for month in months:
            for day in days:
                #checks if it's the end of month for all months except december
                if (month == 2 and day==28)|(month in [4,6,9,11] and day == 30)|(month in [1,3,5,7,8,10] and day == 31):
                    gen = api.search_comments(q=ticker, 
                                  subreddit='wallstreetbets',
                                  after=int(dt.datetime(year, month, day).timestamp()),
                                  before=int(dt.datetime(year, month+1, 1).timestamp()),
                                  filter=['body','author','subreddit']
                                 )
                #checks if it's december
                elif month ==12 and day ==31:
                    gen = api.search_comments(q=ticker, 
                                  subreddit='wallstreetbets',
                                  after=int(dt.datetime(year, month, day).timestamp()),
                                  before=int(dt.datetime(year, 1, 1).timestamp()),
                                  filter=['body','author','subreddit']
                                 )
                else:
                    try:
                        gen = api.search_comments(q=ticker, 
                                  subreddit='wallstreetbets',
                                  after=int(dt.datetime(year, month, day).timestamp()),
                                  before=int(dt.datetime(year, month, day+1).timestamp()),
                                  filter=['body','author','subreddit']
                                 )
                    except:
                        continue
                csv = ticker_reddit_scraper_ends(gen)
                if csv.empty != True:
                    csv.to_csv('csv/'+ticker+'_'+str(year)+'_'+str(month)+'_'+str(day)+'.csv',index=False)#querie reddit for ticker

In [3]:
#rewrites scraper function for r/stocks subreddit
def ticker_reddit_scraper(ticker,year,month):
    #querie reddit for ticker
    gen = api.search_comments(q=ticker, 
                              subreddit='stocks',
                              after=int(dt.datetime(year, month,1).timestamp()),
                              before=int(dt.datetime(year, month+1,1).timestamp()),
                              filter=['body','author','subreddit']
                             )
    #max allowed retrieved responses to ensure not to long
    max_response=310000
    cache=[]
    for i in gen:
        #take only the dictionary from queries
        cache.append(i[-1])
        if len(cache) >= max_response:
            break
    df = pd.DataFrame(cache)
    if df.empty != True:
        try:
            df['created_utc']=pd.to_datetime(df['created_utc'],unit='s').dt.date
        except KeyError:
            return df
        df = df.drop(columns=['created'])
        df.rename(columns={'author':'username','created_utc':'date'}, inplace=True)
        df=df[['date','body','username','subreddit']]
    return df

In [6]:
#creates engine to connect to AWS server
db_string = f"postgresql://postgres:{db_password}@{db_address}:5432/Stockdata"
engine = create_engine(db_string)

In [7]:
#scrapes r/stock by month except december
years = [2021,2020,2019,2018]
months=[1,2,3,4,5,6,7,8,9,10,11]
for ticker in tickers+tickers_done:
    for year in years:
        for month in months:
                csv = ticker_reddit_scraper(ticker,year,month)
                if csv.empty != True:
                    #tries to write AWS if not writes csv
                    try:
                        csv.to_sql(name='comments_posts', 
                              con=engine,
                              if_exists='append',
                              index=False,
                              chunksize=5000
                             )
                    except:
                        csv.to_csv('csv/'+ticker+'_'+str(year)+'_'+str(month)+'_stocks.csv')
                    

C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 502
  warnings.warn("Got non 200 code %s" % response.status_code)


In [8]:
#gets december
years = [2021,2020,2019,2018]
for ticker in tickers+tickers_done:
    for year in years:
        gen = api.search_comments(q=ticker, 
                              subreddit='stocks',
                              after=int(dt.datetime(year, 12,1).timestamp()),
                              before=int(dt.datetime(year+1, 1,1).timestamp()),
                              filter=['body','author','subreddit']
                             )
        cache=[]
        for i in gen:
            #take only the dictionary from queries
            cache.append(i[-1])
        df = pd.DataFrame(cache)
        if df.empty != True:
            try:
                df['created_utc']=pd.to_datetime(df['created_utc'],unit='s').dt.date
            except KeyError:
                df =df
            df = df.drop(columns=['created'])
            df.rename(columns={'author':'username','created_utc':'date'}, inplace=True)
            df=df[['date','body','username','subreddit']]
            if df.empty != True:
                    try:
                        df.to_sql(name='comments_posts', 
                              con=engine,
                              if_exists='append',
                              index=False,
                              chunksize=5000
                             )
                    except:
                        df.to_csv('csv/'+ticker+'_'+str(year)+'_'+str(month)+'_stocks.csv')
                    

In [13]:
#upload file that failed to upload when it was pulled
df = pd.read_csv('csv/re_2020_11_stocks.csv',index_col=0)
df.to_sql(name='comments_posts', 
                              con=engine,
                              if_exists='append',
                              index=False,
                              chunksize=50
                             )